In [1]:
import pandas as pd
import numpy as np
from collections import Counter, defaultdict

In [2]:
train=pd.read_csv('trainSet.csv',index_col=[0])

In [3]:
test=pd.read_csv('testSet.csv',index_col=[0])

In [4]:
def nbc(train,frac):
    rand_train=train.sample(frac=frac,random_state=47)
    return rand_train 

def occurrences(list1):
    no_of_examples = len(list1)
    prob = dict(Counter(list1))
    for key in prob.keys():
        prob[key] = prob[key] / float(no_of_examples)
    return prob
 
def naive_bayes(training, outcome, new_sample):
    classes     = np.unique(outcome)
    rows, cols  = np.shape(training)
    likelihoods = {}
    for cls in classes:
        likelihoods[cls] = defaultdict(list)
  
    class_probabilities = occurrences(outcome)
  
    for cls in classes:
        row_indices = np.where(outcome == cls)[0]
        subset      = training[row_indices, :]
        r, c        = np.shape(subset)
        for j in range(0,c):
            likelihoods[cls][j] += list(subset[:,j])
  
    for cls in classes:
        for j in range(0,cols):
             likelihoods[cls][j] = occurrences(likelihoods[cls][j])  
  
    results = []
    for sample in new_sample:
        cur_sample_result={}
        for cls in classes:
            class_probability = class_probabilities[cls]
            for i in range(0,len(sample)):
                relative_values = likelihoods[cls][i]
                if sample[i] in relative_values.keys():
                    class_probability *= relative_values[sample[i]]
                else:
                    class_probability *= 0
            cur_sample_result[cls] = class_probability
        results.append(cur_sample_result)
    return results

def predict(training, outcome, new_sample):
    t=naive_bayes(training, outcome, new_sample)
    predict=[]
    for i in range(len(t)):
        predict.append(max(t[i], key=t[i].get))
    return predict
    
def accuracy(predict,new_outcome):
    correct = 0
    for i in range(len(new_outcome)):
        if predict[i]==new_outcome[i]:
            correct+=1
    return (correct/float(len(new_outcome)))

def train_accuracy_main():
    # test training sample model
    frac=1
    rand_train=nbc(train,frac)
    training=rand_train.loc[:, train.columns != 'decision'].values
    train_outcome=rand_train['decision'].values
    naive_bayes(training, train_outcome, training)
    predictions = predict(training, train_outcome, training)
    final_accuracy = accuracy(predictions, train_outcome)
    print('Accuracy of the trainSet: {0}'.format(final_accuracy))
train_accuracy_main()

def test_accuracy_main():
    # test testing sample model
    frac=1
    rand_train=nbc(train,frac)
    training=rand_train.loc[:, train.columns != 'decision'].values
    train_outcome=rand_train['decision'].values
    #outcome    = np.asarray((0,1,1,1,0,1,0,1))
    testing = test.loc[:, test.columns != 'decision'].values
    test_outcome=test['decision'].values
    naive_bayes(training, train_outcome, testing)
    predictions = predict(training, train_outcome, testing)
    final_accuracy = accuracy(predictions, test_outcome)
    print('Accuracy of the testset: {0}'.format(final_accuracy))
test_accuracy_main()

Accuracy of the trainSet: 0.7818350324374421
Accuracy of the testset: 0.7472201630837657
